In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score , confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.utils import resample
import pickle
import joblib

In [2]:
df_ = pd.read_pickle("/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/processed/final_processed_data.pkl")

In [3]:
df = df_.copy()

In [4]:
df.drop(df.tail(1000).index,inplace=True)
df

,Name,Parent_full_DNA_Seq,Child_full_DNA_Seq,ParentM,ParentF,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0
...,...,...,...,...,...,...
13995,E13995,CTCCGTCGACGCTTTAGGGCCATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13996,E13996,CTCCGTCGAAGCTTTAGGGACATAGATGGGAGCTCTGGTTCCCGTG...,NaN,NaN,NaN,0
13997,E13997,CTCCGTCGACGCTTTAAGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13998,E13998,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0


In [5]:
# data_0 = df[df['target'] == 0.0]
# data_1 = df[df['target'] == 1.0]

# from sklearn.utils import resample

# data_0 = resample(data_0 ,replace=False ,n_samples=12000 ,random_state=42)
# data_1 = resample(data_1 ,replace=False ,n_samples=12000 ,random_state=42)

# df = pd.concat([data_0,data_1])
# df.reset_index(inplace=True)
# df.drop('index',axis=1,inplace=True)
df

,Name,Parent_full_DNA_Seq,Child_full_DNA_Seq,ParentM,ParentF,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0
...,...,...,...,...,...,...
13995,E13995,CTCCGTCGACGCTTTAGGGCCATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13996,E13996,CTCCGTCGAAGCTTTAGGGACATAGATGGGAGCTCTGGTTCCCGTG...,NaN,NaN,NaN,0
13997,E13997,CTCCGTCGACGCTTTAAGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13998,E13998,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0


In [6]:
df['Parent_full_DNA_Seq'] = df['Parent_full_DNA_Seq'].astype('str')
df['Child_full_DNA_Seq'] = df['Child_full_DNA_Seq'].astype('str')

In [7]:
df_pre = df[['Parent_full_DNA_Seq','Child_full_DNA_Seq','target']]

In [8]:
x = df_pre.drop('target',axis=1)
y = df_pre['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [9]:
x.shape , y.shape

((55784, 2), (55784,))

In [10]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [11]:
kmer_embeddings_c = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/train_vectorizer_np/kmer_embeddings_c.npy')
kmer_embeddings_p = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/train_vectorizer_np/kmer_embeddings_p.npy')

In [12]:
kmer_embeddings_p = np.pad(kmer_embeddings_p, ((0, 0), (0, 1)), 'constant')
for i in range (0 , len(kmer_embeddings_c[0])):
    X_train['child_gene_k_' + str(i)] = [kmer_embeddings_c[j][i] for j in range (0, len(X_train))]
    X_train['parent_gene_k_' + str(i)] = [kmer_embeddings_p[j][i] for j in range (0, len(X_train))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_29132/699562758.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['parent_gene_k_' + str(i)] = [kmer_embeddings_p[j][i] for j in range (0, len(X_train))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_29132/699562758.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['child_gene_k_' + str(i)] = [kmer_embeddings_c[j][i] for j in range (0, len(X_train))]


In [13]:
X_train = X_train.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1)

In [14]:
# param_grid = {
#     'n_estimators': [15000],  # Adjust the values based on your requirements
#     'learning_rate': [0.3],  # Adjust the values based on your requirements
#     'base_estimator__max_depth': [15]  # If you're using DecisionTreeClassifier as base estimator
# }

In [15]:
kmer_embeddings_child_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_child_test.npy')
kmer_embeddings_parent_test = np.load('/Users/0xnrous/Developer/e2e_graduationPROJECT/e2e_graduationPROJECT_DS/data/interim/test_vectorizer_np/kmer_embeddings_parent_test.npy')

In [16]:
for i in range (0, len(kmer_embeddings_child_test[0])):
    X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]
    X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_29132/477141918.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_29132/477141918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]


In [17]:
X_train = X_train.drop(['child_gene_k_64','parent_gene_k_64'],axis=1)

In [18]:
X_test.shape , X_train.shape

((16736, 130), (39048, 128))

In [19]:
X_test.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1,inplace=True)

In [20]:
X_test

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
5874,1020,1052,1153,1136,1136,1137,1038,1068,1157,1164,...,1145,1176,1029,1025,1196,1192,1123,1100,1047,1050
6992,1028,1056,1141,1143,1152,1173,992,1007,1124,1149,...,1147,1159,1069,1077,1159,1132,1167,1117,1138,1122
16861,1066,1091,1153,1171,1213,1196,1027,988,1112,1081,...,1158,1171,1031,1046,1195,1215,1163,1171,1032,1124
1698,0,1136,0,1218,0,1139,0,1046,0,1152,...,0,1208,0,1147,0,1223,0,1221,0,1115
19771,1060,1077,1171,1188,1158,1180,1040,1007,1075,1090,...,1146,1159,1105,1064,1223,1186,1133,1150,1157,1172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13182,1024,999,1122,1158,1161,1164,987,982,1072,1110,...,1219,1177,1056,1054,1190,1177,1193,1176,1108,1105
13424,0,1168,0,1217,0,1201,0,1147,0,1207,...,0,1188,0,1119,0,1185,0,1164,0,1094
10599,992,1037,1111,1142,1163,1187,999,973,1102,1107,...,1200,1182,1093,1059,1192,1167,1129,1175,1112,1143
17685,1035,1095,1163,1130,1170,1159,1004,1002,1131,1165,...,1219,1180,1072,1019,1202,1200,1121,1110,1056,1108


In [21]:
X_train

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
2321,1114,1095,1182,1156,1156,1180,1054,1028,1120,1102,...,1193,1173,1167,1078,1079,1230,1178,1158,1118,1124
8101,1093,1120,1147,1127,1153,1161,1056,1028,1092,1157,...,1238,1129,1120,1066,1059,1202,1221,1140,1148,1132
2754,1012,1049,1195,1117,1162,1153,1004,1058,1111,1098,...,1246,1204,1199,1047,1060,1197,1214,1127,1126,1126
15740,1108,1068,1081,1083,1167,1152,1034,1028,1048,1087,...,1225,1201,1205,1027,1064,1227,1208,1179,1162,1112
2617,1053,1040,1126,1095,1181,1153,1040,1050,1126,1128,...,1213,1116,1148,1107,1044,1192,1190,1140,1135,1135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0,1135,0,1254,0,1203,0,1100,0,1182,...,0,1141,0,1132,0,1176,0,1146,0,1097
12559,0,1103,0,1239,0,1188,0,1144,0,1155,...,0,1153,0,1068,0,1135,0,1214,0,1156
18266,1043,1043,1176,1136,1173,1174,1042,1032,1169,1192,...,1239,1197,1162,1058,1068,1183,1199,1123,1105,1092
860,1001,1044,1130,1151,1187,1145,1019,1040,1137,1104,...,1230,1202,1178,1088,1070,1184,1202,1142,1126,1152


In [22]:
'''
base_estimator = RandomForestClassifier(n_estimators=20, n_jobs=-1)

# Create MultiOutputClassifier
multi_output_clf = MultiOutputClassifier(base_estimator)
'''


# Define hyperparameters for grid search
'''
parameters = {
    'estimator__max_depth': [10, 15, 20],
    'estimator__n_estimators': [100, 300, 500, 1000, 1500],  # Extend the range
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
    'estimator__bootstrap': [True, False]
}

'''

"\nparameters = {\n    'estimator__max_depth': [10, 15, 20],\n    'estimator__n_estimators': [100, 300, 500, 1000, 1500],  # Extend the range\n    'estimator__min_samples_split': [2, 5, 10],\n    'estimator__min_samples_leaf': [1, 2, 4],\n    'estimator__bootstrap': [True, False]\n}\n\n"

# Hyperparameter tuning:


In [23]:
parameters = {
    'estimator__max_depth': [15, 20, 25],  # Slightly higher values
    'estimator__n_estimators': [1000, 1500, 2000],  # A bit more aggressive
    'estimator__min_samples_split': [2, 3, 4],  # Fine-tuning
    'estimator__min_samples_leaf': [1, 2, 3],  # Fine-tuning
    'estimator__bootstrap': [True, False]
}

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [25]:
X_train

array([[0.95213675, 0.49295775, 0.9456    , ..., 0.47963801, 0.90016103,
        0.50326797],
       [0.93418803, 0.56338028, 0.9176    , ..., 0.39819005, 0.92431562,
        0.52941176],
       [0.86495726, 0.36338028, 0.956     , ..., 0.33936652, 0.90660225,
        0.50980392],
       ...,
       [0.89145299, 0.34647887, 0.9408    , ..., 0.32126697, 0.88969404,
        0.39869281],
       [0.85555556, 0.34929577, 0.904     , ..., 0.40723982, 0.90660225,
        0.59477124],
       [0.89487179, 0.38028169, 0.92      , ..., 0.19457014, 0.9041868 ,
        0.52287582]])

In [26]:
# train_x = X_train.values
# test_x = X_test.values

In [27]:
# grid_search = GridSearchCV(estimator=ada_boost, param_grid=param_grid, scoring='accuracy', cv=5)
print(y_train.shape)

(39048,)


In [28]:
y_train = y_train.reshape(-1, 1)


In [29]:
with open('../models/model_xgb_clf_hyperparameter.pkl', 'rb') as f:
  model_clf = pickle.load(f)

In [31]:
scorer = make_scorer(fbeta_score, beta=2, average='macro')

# Create a GridSearchCV model
# model_clf = GridSearchCV(estimator=multi_output_clf, param_grid=parameters, scoring=scorer, n_jobs=-1, cv=2, verbose=3)

# Fit the model to the training data
model_clf.fit(X_train, y_train)
# 63m 50.4s

Fitting 2 folds for each of 270 candidates, totalling 540 fits
[CV 1/2] END estimator__bootstrap=True, estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100;, score=0.686 total time=   5.3s
[CV 2/2] END estimator__bootstrap=True, estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100;, score=0.696 total time=   5.4s
[CV 1/2] END estimator__bootstrap=True, estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=5, estimator__n_estimators=100;, score=0.685 total time=   6.1s
[CV 2/2] END estimator__bootstrap=True, estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=5, estimator__n_estimators=100;, score=0.689 total time=   6.5s
[CV 2/2] END estimator__bootstrap=True, estimator__max_depth=10, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=300;, score=0.707 total time=

GridSearchCV(cv=2,
             estimator=MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=20,
                                                                              n_jobs=-1)),
             n_jobs=-1,
             param_grid={'estimator__bootstrap': [True, False],
                         'estimator__max_depth': [10, 15, 20],
                         'estimator__min_samples_leaf': [1, 2, 4],
                         'estimator__min_samples_split': [2, 5, 10],
                         'estimator__n_estimators': [100, 300, 500, 1000,
                                                     1500]},
             scoring=make_scorer(fbeta_score, beta=2, average=macro),
             verbose=3)

In [32]:
y_pred_clf = model_clf.predict(X_test)

In [33]:
y_pred_clf

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [34]:
accuracy = accuracy_score(y_test, y_pred_clf)
precision = precision_score(y_test, y_pred_clf)
recall = recall_score(y_test, y_pred_clf)
f1 = f1_score(y_test, y_pred_clf)
conf_matrix = confusion_matrix(y_test, y_pred_clf)

In [35]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("\n")

Accuracy: 77.50%
Precision: 81.52%
Recall: 51.35%
F1 Score: 63.00%
Confusion Matrix:
[[9765  727]
 [3038 3206]]




In [ ]:
# print(f"Accuracy: {accuracy * 100:.2f}%")
# print(f"Precision: {precision * 100:.2f}%")
# print(f"Recall: {recall * 100:.2f}%")
# print(f"F1 Score: {f1 * 100:.2f}%")
# print("Confusion Matrix:")
# print(conf_matrix)
# print("\n")

# test 1 : with fitting time of 63m 50.4s
# # Accuracy: 78.63%
# # Precision: 82.36%
# # Recall: 54.37%
# # F1 Score: 65.50%
# # Confusion Matrix:
# # [[9765  727]
# #  [2849 3395]]

# test 2 : with fitting time of 63m 50.4s with same parameters

# Accuracy: 77.50%
# Precision: 81.52%
# Recall: 51.35%
# F1 Score: 63.00%
# Confusion Matrix:
# [[9765  727]
#  [3038 3206]]


In [ ]:
#saving Model
# with open('model_xgb_clf_hyperparameter.pkl' , 'wb') as f:
#   pickle.dump(model_clf, f)